In [ ]:
!pip install qutip

In [ ]:
!pip install krotov

In [ ]:
import sys
import os
import qutip
import numpy as np
import scipy
import matplotlib
import matplotlib.pylab as plt
import krotov
from scipy.fftpack import fft
from scipy.interpolate import interp1d
from qutip.control import *

from scipy.stats import unitary_group
import math as mt

In [ ]:
# Pauli matrices
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1.j], [1.j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

Isigma_z = np.array([[1, 0, 0, 0],
           [0, -1, 0, 0],
           [0, 0, 1, 0],
           [0, 0, 0, -1]])

sigma_zI =np.array( [[1, 0, 0, 0],
           [0, 1, 0, 0],
           [0, 0, -1, 0],
           [0, 0, 0, -1]])

Isigma_x = np.array([[0, 1, 0, 0],
           [1, 0, 0, 0],
           [0, 0, 0, 1],
           [0, 0, 1, 0]])

sigma_xI = np.array([[0, 0, 1, 0],
           [0, 0, 0, 1],
           [1, 0, 0, 0],
           [0, 1, 0, 0]])
Isigma_y = np.array([[0, -1.j, 0, 0],
           [1.j, 0, 0, 0],
           [0, 0, 0, -1.j],
           [0, 0, 1.j, 0]])

sigma_yI = np.array([[0, 0, -1.j, 0],
           [0, 0, 0, -1.j],
           [1.j, 0, 0, 0],
           [0, 1.j, 0, 0]])

sigma_x_x = np.array([[0, 0, 0, 1],
           [0, 0, 1, 0],
           [0, 1, 0, 0],
           [1, 0, 0, 0]])

sigma_y_y = np.array([[0, 0, 0, -1],
           [0, 0, 1, 0],
           [0, 1, 0, 0],
           [-1, 0, 0, 0]])

sigma_z_z = np.array([[1, 0, 0, 0],
           [0, -1, 0, 0],
           [0, 0, -1, 0],
           [0, 0, 0, 1]])


In [ ]:
def guess_control1(t, args , ampl0=0.2):
    return ampl0 * krotov.shapes.flattop(
        t, t_start=0, t_stop=300, t_rise=0.3, func="blackman"
    )
def guess_control2(t , args,ampl0=0.2):
    return ampl0 * krotov.shapes.flattop(
        t, t_start=0, t_stop=300, t_rise=0.3, func="blackman"
    )

In [ ]:
def hamiltonian(omega1=1 ,omega2=1, ampl0=0.2, J=0.01  ):
    H0 = qutip.Qobj((-0.5) *( omega1 * Isigma_z +  omega2 * sigma_zI )+J* (sigma_z_z))
    H1 = qutip.Qobj(Isigma_x)
    H2 = qutip.Qobj(sigma_xI)


    return [H0,[H1,guess_control1],[H2,guess_control2]]


H = hamiltonian()

In [ ]:
tlist = np.linspace(0, 300, 3000)

In [ ]:
def logical_basis(H):
    H0 = qutip.Qobj((-0.5) *(1 * Isigma_z +  1 * sigma_zI ))
    eigenvals, eigenvecs = scipy.linalg.eig(H0.full())
    ndx = np.argsort(eigenvals.real)
    E = eigenvals[ndx].real
    V = eigenvecs[:, ndx]
    psi00 = qutip.Qobj(V[:, 0])
    psi01 = qutip.Qobj(V[:, 1])
    psi10 = qutip.Qobj(V[:, 2])
    psi11 = qutip.Qobj(V[:, 3])
    w01 = E[1] - E[0]  # Transition energy between states
    print("Energy of qubit transition is %.3f" % w01)
    return psi00, psi01, psi10, psi11

psi00, psi01, psi10, psi11 = logical_basis(H)

Energy of qubit transition is 1.000


In [ ]:
proj00 = qutip.ket2dm(psi00)
proj01 = qutip.ket2dm(psi01)
proj10 = qutip.ket2dm(psi10)
proj11 = qutip.ket2dm(psi11)

In [ ]:

RandU_2Q = []
N = 100

for i in range(N):
    teta = np.random.uniform(0, np.pi)
    phi = np.random.uniform(0, 2*np.pi)
    landa = np.random.uniform(0, 2*np.pi)
    gama = np.random.uniform(0, 2*np.pi)
    RandU_2Q.append(CRU(teta, phi, landa, gama))


In [ ]:
RandU_2Q

[array([[ 1.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  1.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
         -0.86232933+0.50404216j, -0.01182272-0.04679586j],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.03843368+0.02919728j,  0.80137474-0.59621213j]]),
 array([[ 1.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  1.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
         -0.22357571-0.97444447j,  0.00655555-0.02071014j],
        [ 0.        +0.j        ,  0.        +0.j        ,
         -0.01933426-0.00990309j,  0.53557483+0.84420834j]]),
 array([[ 1.        +0.j        ,  0.       

In [ ]:
basis = [psi00, psi01, psi10, psi11]
basis

[Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
 Qobj data =
 [[1.]
  [0.]
  [0.]
  [0.]],
 Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
 Qobj data =
 [[0.]
  [1.]
  [0.]
  [0.]],
 Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
 Qobj data =
 [[0.]
  [0.]
  [1.]
  [0.]],
 Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
 Qobj data =
 [[0.]
  [0.]
  [0.]
  [1.]]]

In [ ]:

objectives=[0]*100
for i in range(100):
  objectives[i] = krotov.gate_objectives(
      basis_states=[psi00, psi01, psi10, psi11], gate=RandU_2Q[i], H=H
  )



def S(t):
    """Scales the Krotov methods update of the pulse value at the time t"""
    return krotov.shapes.flattop(
        t, t_start=0, t_stop=300, t_rise=0.3, t_fall=0.3, func='blackman'
   )

#pulse_options = {H[1][1]: dict(lambda_a=1, update_shape=S)}

pulse_options = {
    H[1][1]: dict(lambda_a=1, update_shape=S),
    H[2][1]: dict(lambda_a=1, update_shape=S)
}

In [ ]:
results=[]
for i in range(5):

  opt_result = krotov.optimize_pulses(
      objectives[i],
      pulse_options=pulse_options,
      tlist=tlist,
      propagator=krotov.propagators.expm,
      chi_constructor=krotov.functionals.chis_sm,
      info_hook=krotov.info_hooks.print_table(J_T=krotov.functionals.J_T_sm),
      check_convergence=krotov.convergence.Or(
          krotov.convergence.value_below('1e-3', name='J_T'),
          krotov.convergence.check_monotonic_error,
      ),
      store_all_pulses=True,
  )

  results.append(opt_result)


  #j_f=[]
  #j_f.append(opt_result.info_vals)
#print(j_f)


iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.29e-01    0.00e+00   9.29e-01        n/a        n/a    46
1     1.47e-01    1.74e-01   3.22e-01  -7.82e-01  -6.08e-01   100
2     6.69e-02    3.94e-02   1.06e-01  -8.06e-02  -4.11e-02    98
3     5.07e-02    8.06e-03   5.88e-02  -1.62e-02  -8.14e-03   100
4     3.46e-02    7.99e-03   4.26e-02  -1.62e-02  -8.17e-03    97
5     2.59e-02    4.32e-03   3.02e-02  -8.66e-03  -4.33e-03   103
6     1.60e-02    4.92e-03   2.10e-02  -9.87e-03  -4.96e-03   104
7     1.21e-02    1.95e-03   1.41e-02  -3.91e-03  -1.95e-03   108
8     8.98e-03    1.58e-03   1.06e-02  -3.15e-03  -1.58e-03   107
9     5.79e-03    1.59e-03   7.38e-03  -3.19e-03  -1.60e-03   106
10    3.71e-03    1.04e-03   4.75e-03  -2.07e-03  -1.04e-03   107
11    2.48e-03    6.15e-04   3.10e-03  -1.23e-03  -6.15e-04   102
12    1.59e-03    4.48e-04   2.04e-03  -8.95e-04  -4.47e-04   102
13    9.93e-04    2.98e-04   1.29e-03  -5.97e-04  -2.98e-04   102
iter.     

In [ ]:
results2=[]
for i in range(5,10):

  opt_result = krotov.optimize_pulses(
      objectives[i],
      pulse_options=pulse_options,
      tlist=tlist,
      propagator=krotov.propagators.expm,
      chi_constructor=krotov.functionals.chis_sm,
      info_hook=krotov.info_hooks.print_table(J_T=krotov.functionals.J_T_sm),
      check_convergence=krotov.convergence.Or(
          krotov.convergence.value_below('1e-3', name='J_T'),
          krotov.convergence.check_monotonic_error,
      ),
      store_all_pulses=True,
  )

  results2.append(opt_result)





iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.60e-01    0.00e+00   9.60e-01        n/a        n/a    47
1     2.39e-01    1.35e-01   3.73e-01  -7.21e-01  -5.86e-01    98
2     6.42e-02    8.27e-02   1.47e-01  -1.75e-01  -9.20e-02   100
3     3.20e-02    1.59e-02   4.79e-02  -3.22e-02  -1.62e-02   100
4     2.01e-02    5.91e-03   2.61e-02  -1.19e-02  -5.95e-03   106
5     1.11e-02    4.52e-03   1.56e-02  -9.06e-03  -4.54e-03   107
6     4.13e-03    3.45e-03   7.59e-03  -6.95e-03  -3.50e-03   106
7     3.02e-03    5.53e-04   3.58e-03  -1.11e-03  -5.57e-04   105
8     1.70e-03    6.60e-04   2.36e-03  -1.32e-03  -6.64e-04   102
9     1.53e-03    8.39e-05   1.61e-03  -1.67e-04  -8.35e-05   101
10    1.17e-03    1.81e-04   1.35e-03  -3.63e-04  -1.81e-04   100
11    1.08e-03    4.38e-05   1.12e-03  -8.79e-05  -4.41e-05   101
12    9.29e-04    7.56e-05   1.00e-03  -1.51e-04  -7.54e-05   101
iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.84

In [ ]:
results3=[]
for i in range(10,15):

  opt_result = krotov.optimize_pulses(
      objectives[i],
      pulse_options=pulse_options,
      tlist=tlist,
      propagator=krotov.propagators.expm,
      chi_constructor=krotov.functionals.chis_sm,
      info_hook=krotov.info_hooks.print_table(J_T=krotov.functionals.J_T_sm),
      check_convergence=krotov.convergence.Or(
          krotov.convergence.value_below('1e-3', name='J_T'),
          krotov.convergence.check_monotonic_error,
      ),
      store_all_pulses=True,
  )

  results3.append(opt_result)


  #j_f=[]
  #j_f.append(opt_result.info_vals)
#print(j_f)


iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.17e-01    0.00e+00   9.17e-01        n/a        n/a    42
1     4.12e-01    1.38e-01   5.50e-01  -5.05e-01  -3.67e-01    87
2     7.51e-02    1.49e-01   2.24e-01  -3.37e-01  -1.88e-01    87
3     2.71e-02    2.36e-02   5.08e-02  -4.80e-02  -2.43e-02    87
4     2.59e-02    6.06e-04   2.65e-02  -1.21e-03  -6.07e-04    88
5     2.32e-02    1.36e-03   2.46e-02  -2.72e-03  -1.36e-03    89
6     2.10e-02    1.10e-03   2.21e-02  -2.21e-03  -1.10e-03    90
7     1.85e-02    1.27e-03   1.97e-02  -2.54e-03  -1.27e-03    88
8     1.55e-02    1.47e-03   1.70e-02  -2.93e-03  -1.47e-03    86
9     1.22e-02    1.67e-03   1.39e-02  -3.34e-03  -1.67e-03    87
10    8.86e-03    1.66e-03   1.05e-02  -3.32e-03  -1.66e-03    87
11    6.12e-03    1.37e-03   7.49e-03  -2.73e-03  -1.37e-03    86
12    4.10e-03    1.01e-03   5.11e-03  -2.02e-03  -1.01e-03    86
13    2.74e-03    6.80e-04   3.42e-03  -1.36e-03  -6.82e-04    86
14    1.83

In [ ]:
results4=[]
for i in range(15,20):

  opt_result = krotov.optimize_pulses(
      objectives[i],
      pulse_options=pulse_options,
      tlist=tlist,
      propagator=krotov.propagators.expm,
      chi_constructor=krotov.functionals.chis_sm,
      info_hook=krotov.info_hooks.print_table(J_T=krotov.functionals.J_T_sm),
      check_convergence=krotov.convergence.Or(
          krotov.convergence.value_below('1e-3', name='J_T'),
          krotov.convergence.check_monotonic_error,
      ),
      store_all_pulses=True,
  )

  results4.append(opt_result)


  #j_f=[]
  #j_f.append(opt_result.info_vals)
#print(j_f)


iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.51e-01    0.00e+00   9.51e-01        n/a        n/a    30
1     8.25e-01    4.17e-02   8.66e-01  -1.26e-01  -8.42e-02    64
2     5.38e-01    1.09e-01   6.46e-01  -2.87e-01  -1.78e-01    64
3     3.49e-01    8.61e-02   4.35e-01  -1.89e-01  -1.03e-01    64
4     2.26e-01    5.85e-02   2.84e-01  -1.23e-01  -6.43e-02    68
5     1.59e-01    3.25e-02   1.92e-01  -6.66e-02  -3.42e-02    67
6     1.19e-01    1.98e-02   1.39e-01  -4.04e-02  -2.05e-02    66
7     9.49e-02    1.18e-02   1.07e-01  -2.40e-02  -1.21e-02    66
8     7.72e-02    8.77e-03   8.60e-02  -1.77e-02  -8.92e-03    66
9     6.31e-02    7.02e-03   7.01e-02  -1.41e-02  -7.10e-03    66
10    5.09e-02    6.04e-03   5.70e-02  -1.21e-02  -6.10e-03    63
11    4.05e-02    5.19e-03   4.57e-02  -1.04e-02  -5.24e-03    65
12    3.19e-02    4.30e-03   3.62e-02  -8.63e-03  -4.33e-03    69
13    2.50e-02    3.43e-03   2.84e-02  -6.89e-03  -3.46e-03    68
14    1.96

In [ ]:
results5=[]
for i in range(20,25):

  opt_result = krotov.optimize_pulses(
      objectives[i],
      pulse_options=pulse_options,
      tlist=tlist,
      propagator=krotov.propagators.expm,
      chi_constructor=krotov.functionals.chis_sm,
      info_hook=krotov.info_hooks.print_table(J_T=krotov.functionals.J_T_sm),
      check_convergence=krotov.convergence.Or(
          krotov.convergence.value_below('1e-3', name='J_T'),
          krotov.convergence.check_monotonic_error,
      ),
      store_all_pulses=True,
  )

  results5.append(opt_result)


  #j_f=[]
  #j_f.append(opt_result.info_vals)
#print(j_f)


iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.84e-01    0.00e+00   9.84e-01        n/a        n/a    43
1     3.47e-01    6.36e-02   4.11e-01  -6.37e-01  -5.73e-01    91
2     8.54e-02    1.20e-01   2.05e-01  -2.62e-01  -1.42e-01    92
3     5.48e-02    1.52e-02   7.00e-02  -3.07e-02  -1.54e-02    91
4     1.55e-02    1.94e-02   3.49e-02  -3.92e-02  -1.98e-02    90
5     8.38e-03    3.57e-03   1.19e-02  -7.17e-03  -3.60e-03    91
6     5.85e-03    1.26e-03   7.12e-03  -2.53e-03  -1.26e-03    90
7     4.67e-03    5.91e-04   5.26e-03  -1.19e-03  -5.95e-04    91
8     3.98e-03    3.44e-04   4.32e-03  -6.90e-04  -3.45e-04    91
9     3.29e-03    3.40e-04   3.63e-03  -6.83e-04  -3.43e-04    91
10    2.34e-03    4.74e-04   2.82e-03  -9.49e-04  -4.75e-04    91
11    1.25e-03    5.47e-04   1.80e-03  -1.09e-03  -5.46e-04    90
12    5.63e-04    3.44e-04   9.08e-04  -6.88e-04  -3.43e-04    90
iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.21

In [ ]:
results6=[]
for i in range(25,30):

  opt_result = krotov.optimize_pulses(
      objectives[i],
      pulse_options=pulse_options,
      tlist=tlist,
      propagator=krotov.propagators.expm,
      chi_constructor=krotov.functionals.chis_sm,
      info_hook=krotov.info_hooks.print_table(J_T=krotov.functionals.J_T_sm),
      check_convergence=krotov.convergence.Or(
          krotov.convergence.value_below('1e-3', name='J_T'),
          krotov.convergence.check_monotonic_error,
      ),
      store_all_pulses=True,
  )

  results6.append(opt_result)


  #j_f=[]
  #j_f.append(opt_result.info_vals)
#print(j_f)


iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.78e-01    0.00e+00   9.78e-01        n/a        n/a    29
1     1.99e-01    8.71e-02   2.86e-01  -7.79e-01  -6.92e-01    64
2     1.40e-01    2.90e-02   1.69e-01  -5.89e-02  -3.00e-02    62
3     5.59e-02    4.07e-02   9.66e-02  -8.37e-02  -4.30e-02    63
4     1.88e-02    1.82e-02   3.70e-02  -3.71e-02  -1.89e-02    63
5     1.44e-02    2.22e-03   1.66e-02  -4.44e-03  -2.22e-03    64
6     1.07e-02    1.83e-03   1.25e-02  -3.66e-03  -1.83e-03    64
7     5.48e-03    2.59e-03   8.07e-03  -5.22e-03  -2.63e-03    64
8     4.39e-03    5.45e-04   4.94e-03  -1.09e-03  -5.43e-04    64
9     3.27e-03    5.57e-04   3.83e-03  -1.12e-03  -5.63e-04    64
10    2.06e-03    6.03e-04   2.67e-03  -1.21e-03  -6.09e-04    64
11    1.61e-03    2.28e-04   1.83e-03  -4.57e-04  -2.28e-04    64
12    1.00e-03    3.01e-04   1.30e-03  -6.05e-04  -3.04e-04    63
13    7.28e-04    1.36e-04   8.65e-04  -2.73e-04  -1.36e-04    64
iter.     

In [ ]:
results7=[]
for i in range(30,35):

  opt_result = krotov.optimize_pulses(
      objectives[i],
      pulse_options=pulse_options,
      tlist=tlist,
      propagator=krotov.propagators.expm,
      chi_constructor=krotov.functionals.chis_sm,
      info_hook=krotov.info_hooks.print_table(J_T=krotov.functionals.J_T_sm),
      check_convergence=krotov.convergence.Or(
          krotov.convergence.value_below('1e-3', name='J_T'),
          krotov.convergence.check_monotonic_error,
      ),
      store_all_pulses=True,
  )

  results7.append(opt_result)


  #j_f=[]
  #j_f.append(opt_result.info_vals)
#print(j_f)


iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.74e-01    0.00e+00   9.74e-01        n/a        n/a    48
1     3.62e-01    1.03e-01   4.65e-01  -6.12e-01  -5.09e-01    99
2     4.64e-02    1.42e-01   1.88e-01  -3.15e-01  -1.74e-01   100
3     1.65e-02    1.49e-02   3.14e-02  -2.99e-02  -1.51e-02    99
4     1.09e-02    2.76e-03   1.37e-02  -5.55e-03  -2.79e-03   104
5     7.91e-03    1.50e-03   9.42e-03  -3.03e-03  -1.53e-03   101
6     1.36e-03    3.26e-03   4.63e-03  -6.55e-03  -3.29e-03   100
7     1.05e-03    1.55e-04   1.21e-03  -3.11e-04  -1.56e-04   101
8     6.43e-04    2.04e-04   8.47e-04  -4.08e-04  -2.04e-04   101
iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.76e-01    0.00e+00   9.76e-01        n/a        n/a    47
1     4.71e-01    8.80e-02   5.59e-01  -5.05e-01  -4.17e-01   103
2     8.67e-02    1.66e-01   2.53e-01  -3.84e-01  -2.18e-01   102
3     1.23e-02    3.65e-02   4.88e-02  -7.44e-02  -3.79e-02   103
4     1.29

In [ ]:
results8=[]
for i in range(35,40):

  opt_result = krotov.optimize_pulses(
      objectives[i],
      pulse_options=pulse_options,
      tlist=tlist,
      propagator=krotov.propagators.expm,
      chi_constructor=krotov.functionals.chis_sm,
      info_hook=krotov.info_hooks.print_table(J_T=krotov.functionals.J_T_sm),
      check_convergence=krotov.convergence.Or(
          krotov.convergence.value_below('1e-3', name='J_T'),
          krotov.convergence.check_monotonic_error,
      ),
      store_all_pulses=True,
  )

  results8.append(opt_result)


  #j_f=[]
  #j_f.append(opt_result.info_vals)
#print(j_f)


iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     8.90e-01    0.00e+00   8.90e-01        n/a        n/a    39
1     6.32e-01    8.82e-02   7.20e-01  -2.58e-01  -1.70e-01    84
2     3.66e-01    1.14e-01   4.81e-01  -2.66e-01  -1.52e-01    91
3     2.17e-01    7.02e-02   2.87e-01  -1.49e-01  -7.88e-02    86
4     1.41e-01    3.68e-02   1.78e-01  -7.58e-02  -3.90e-02    83
5     8.89e-02    2.57e-02   1.15e-01  -5.24e-02  -2.67e-02    85
6     6.22e-02    1.32e-02   7.54e-02  -2.67e-02  -1.35e-02    84
7     4.45e-02    8.77e-03   5.33e-02  -1.77e-02  -8.95e-03    83
8     3.53e-02    4.58e-03   3.99e-02  -9.21e-03  -4.64e-03    83
9     2.87e-02    3.28e-03   3.20e-02  -6.60e-03  -3.32e-03    88
10    2.42e-02    2.26e-03   2.64e-02  -4.53e-03  -2.27e-03    92
11    2.04e-02    1.88e-03   2.23e-02  -3.76e-03  -1.88e-03    91
12    1.69e-02    1.74e-03   1.87e-02  -3.49e-03  -1.75e-03    93
13    1.38e-02    1.56e-03   1.53e-02  -3.13e-03  -1.57e-03    93
14    1.08

In [ ]:
results9=[]
for i in range(40,45):

  opt_result = krotov.optimize_pulses(
      objectives[i],
      pulse_options=pulse_options,
      tlist=tlist,
      propagator=krotov.propagators.expm,
      chi_constructor=krotov.functionals.chis_sm,
      info_hook=krotov.info_hooks.print_table(J_T=krotov.functionals.J_T_sm),
      check_convergence=krotov.convergence.Or(
          krotov.convergence.value_below('1e-3', name='J_T'),
          krotov.convergence.check_monotonic_error,
      ),
      store_all_pulses=True,
  )

  results9.append(opt_result)


  #j_f=[]
  #j_f.append(opt_result.info_vals)
#print(j_f)


iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.84e-01    0.00e+00   9.84e-01        n/a        n/a    42
1     3.75e-01    8.31e-02   4.58e-01  -6.10e-01  -5.27e-01    88
2     4.83e-02    1.46e-01   1.94e-01  -3.26e-01  -1.81e-01    89
3     6.12e-03    2.07e-02   2.68e-02  -4.22e-02  -2.15e-02    89
4     5.21e-03    4.59e-04   5.66e-03  -9.19e-04  -4.60e-04    88
5     2.36e-03    1.41e-03   3.77e-03  -2.84e-03  -1.43e-03    89
6     1.93e-03    2.15e-04   2.15e-03  -4.32e-04  -2.17e-04    93
7     1.29e-03    3.18e-04   1.61e-03  -6.41e-04  -3.23e-04    92
8     1.09e-03    1.00e-04   1.19e-03  -2.01e-04  -1.01e-04    91
9     8.37e-04    1.25e-04   9.61e-04  -2.52e-04  -1.27e-04    91
iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     9.12e-01    0.00e+00   9.12e-01        n/a        n/a    43
1     6.52e-01    6.58e-02   7.18e-01  -2.60e-01  -1.94e-01    90
2     4.40e-01    9.30e-02   5.33e-01  -2.12e-01  -1.19e-01    92
3     3.00